### Science

### ML Engineering

In [ ]:
from sqlalchemy import Column, Integer, String

In [ ]:
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, "id", primary_key=True)
    username = Column(String, "username")

In [ ]:
from metaflow import resources

In [ ]:
func = lambda x: return x**2

### AI

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def by_column_parallelism(inputs, weights):
    n_cols = weights.shape[-1]
    
    w1, w2 = weights[:, :n_cols//2], weights[:, n_cols//2:]
    
    out1 = inputs @ w1
    out2 = inputs @ w2
    
    return torch.cat([out1, out2], dim=-1)

In [ ]:
step 1: split model
step 2: split batch
step 3: forward backward
step 4: compute average graien
step 5: update all
step 6

In [ ]:
def by_row_parallelism(inputs, weights):
    n_cols = weights.shape[-1]
    
    x1, x2 = inputs[:, :n_cols//2], inputs[:, n_cols//2:]
    w1, w2 = weights[:n_cols//2, :], weights[n_cols//2:, :]
    
    out1 = x1 @ w1
    out2 = x2 @ w2
    
    return out1 + out2

In [ ]:
step 1: normalize the loss
step 2: compute the gradient with respect to the normalized loss
stepm 3: 

In [ ]:
class Model(nn.Module):
    def forward(self, x, labels):
        output = self.net(x)
        loss = loss_func(output, labels)
        return output, loss

In [ ]:
outputs, loss

In [ ]:
class BatchNorm(nn.Module):
    def __init__(self, mom, eps):
        super().__init__()
        self.mom, self.eps = mom, eps
        self.adds = nn.Parameter(torch.zeros(1))
        self.mults = nn.Parameter(torch.ones(1))
        self.register_buffer("means", torch.zeros(1))
        self.register_buffer("vars", torch.ones(1))
    
    def update_stats(self, x):
        mean = x.mean(keepdim=True, dim=-1)
        var = x.var(keepdim=True, dim=-1)
        self.means.lerp_(mean, self.mom)
        self.vars.lerp_(var, self.mom)
        
        return mean, var
    
    def forward(self, x):
        with torch.no_grad():
            mean, var = self.update_stats(x)
        
        x = (x-mean)/(var + self.eps)
        x = self.mults*x + self.adds
        return x

In [ ]:
from torch.autograd import Function

In [ ]:
class MultiplyConstant(Function):
    @staticmethod
    def forward(ctx, x):
        return x
    
    @staticmethod
    def backward(ctx, grad):
        return grad + 1

In [ ]:
from ray import tune

In [ ]:
def optimize_func(params):
    x, y = params["x"], params["y"]
    score = objective(x, y)
    tune.report(score=score)

In [ ]:
result = tune.run(
    optimize_func,
    config=config
)

In [ ]:
torch.clamp_min(x, 2)

In [ ]:
import torch.distributed as dist

In [ ]:
dist.broadcast(x, src=0)

In [ ]:
from einops import repeat

In [ ]:
output = repeat(x, "h w -> h w n", n=2)

In [ ]:
for _ in range(N_EPISODE):
    state, _ = env.reset()
    done = True
    
    while done:
        preds = model(state)
        action = torch.argmax(preds, dim=-1)
        predicted_reward = preds[action]
        
        new_state, reward, done, truncated, info = env.step(action)
        next_preds = model(next_state)
        
        if done is False:
            highest_next_pred = torch.argmax(next_preds, dim=-1)
            state = next_state
        else:
            highest_next_pred = 0
        
        target_reward = reward + highest_next_pred
        
        optimizer.zero_grad()
        
        loss = loss_func(predicted_reward, target_reward)
        loss.backward()
        
        optimizer.step()

In [ ]:
flat_func = nn.Flatten(start_dim=1, end_dim=3 )

In [ ]:
dist.recv()

In [ ]:
nn.parallel.replicate()

In [ ]:
dist.get_world_size()

In [ ]:
dist.isend()

In [ ]:
dist.send()

In [ ]:
dist.get_rank()

In [ ]:
dist.reduce()

In [ ]:
step 1: normalize the loss
step 2: compute the gradient with respect to the normalized loss
step 3: accumulate the gradient from all machines
step 4: update with respect to the accumulated gradient if nnnn, else cstep 1